In [1]:
# imports
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
def espaco(linhas):
  return np.zeros((linhas, linhas))

In [3]:
def pega_individuo_nulo(matrix):
    shape = np.shape(matrix)

    x_aleatorio = random.randint(0, shape[0]-1)
    y_aleatorio = random.randint(0, shape[0]-1)

    individuo = matrix[x_aleatorio][y_aleatorio]

    if individuo == 0:
        matrix[x_aleatorio][y_aleatorio] = 1
    else:
        pega_individuo_nulo(matrix)

    return matrix


In [4]:
def individuos(matrix, num_individuos):
    for i in range(0, num_individuos):
        pega_individuo_nulo(matrix)
    return matrix

In [5]:
def vizinhanca(matriz, linha, coluna):
    linhas, colunas = matriz.shape
    vizinhos = []

    # Define as coordenadas relativas dos vizinhos de Moore
    coords_relativas = [(i, j) for i in range(-1, 2) for j in range(-1, 2) if (i != 0 or j != 0)]

    for dx, dy in coords_relativas:
        nova_linha, nova_coluna = linha + dx, coluna + dy
        if 0 <= nova_linha < linhas and 0 <= nova_coluna < colunas:
            vizinhos.append(matriz[nova_linha, nova_coluna])

    return (0, (linha, coluna), vizinhos)

In [ ]:
def vizinhos_adjacente(matriz, vertice):
    linhas, colunas = matriz.shape
    vizinhos_info = []

    for i in range(linhas):
        for j in range(colunas):
            vizinhos = np.where(matriz[i] == 1)[0]
            vizinhos_info.append([0, i, j] + list(vizinhos))

    return np.array(vizinhos_info), vertice

In [ ]:

def infectar(populacao, numero):
    indices_suscetiveis = np.where(populacao[:, 0] == 0)[0]
    indices_infectar = np.random.choice(indices_suscetiveis, size=numero, replace=False)
    populacao[indices_infectar, 0] = 1
    return populacao

In [ ]:
def ruido(populacao, numero, prob):
    indices_ruido = np.random.choice(populacao.shape[0], size=numero, replace=False)
    for i in indices_ruido:
        if random.random() < prob:
            populacao[i, 0] = 1 - populacao[i, 0]  # Inverte o estado de infecção
    return populacao

In [ ]:
def atualizar(populacao, numero, prob):

    nova_matriz = populacao.copy()
    for i in range(populacao.shape[0]):
        if populacao[i, 0] == 0:  # Apenas para indivíduos suscetíveis
            vizinhos_infectados = np.sum(populacao[populacao[i, 3:11], 0] == 1)
            if vizinhos_infectados == numero and np.random.rand() < prob:
                nova_matriz[i, 0] = 1  # Indivíduo suscetível foi infectado
    return nova_matriz


In [ ]:
def gravar(populacao, nome_arquivo):
    np.savetxt(f"{nome_arquivo}.txt", populacao[:, 1:], fmt='%d', delimiter=' ')


In [6]:
matrix = espaco(10)
matrix_com_individuos = individuos(matrix, 5)
populacao_info, max_vizinhos = vizinhos_adjacente(matrix_com_individuos)
populacao_info = infectar(populacao_info, 5)


n_epocas = 50

for tempo in range(1, n_epocas + 1):
    populacao_info = atualizar(populacao_info, 2, 0.05)

    if tempo % 5 == 0:
        gravar(populacao_info, f"estado_{tempo}")

    populacao_info = ruido(populacao_info, 10, 0.005)